In [4]:
import requests
from openai import OpenAI
import pandas as pd
import re

In [5]:
import pickle

with open('nvidia_NV-Embed-v2.pkl', 'rb') as f:
    data = pickle.load(f)

In [12]:
Errors = 0
page_check = 0
tokens = 0
top_k = 10
correct = 0
failure = 0
counter = 0
results = []

for d in data:
    query = d['Q']
    answer = d['A']
    retrived_pages = []
    retrived_indexes = d['question_retrived_page_rank'][:top_k]
    retrived_indexes = sorted([list(retrived_indexes[u].keys())[0] for u in range(len(retrived_indexes))])
    
    for x in data:
        if(int(x['page']) in retrived_indexes):
            # retrived_pages += f"This is page {int(x['page'])}: " + x['content'] + "\n"
            retrived_pages.append({"page": x['page'], "content": x['content']})


    # rerank
    try:
        client = OpenAI(api_key='OPENAI_API_KEY_PLACEHOLDER')
        r = client.chat.completions.create(
            messages=[
                {
                "role": "system", "content": "You are a math expert",
                "content": "user", "content": f"I am giving you a question and a some pages in the textbook. The question may or may not be based on one of the pages. Return ONLY a JSON list of page number such that the one coming first is most relevant to the question, and the last is least relevant. You MUST RETURN ONLY JSON LIST. I am giving you {top_k} pages, and therefore expect a list of the same length. Pages:" + str(retrived_pages) + ", Question:" + query
                }
            ],
            temperature = 0,
            model="gpt-4o-mini",
        )
        res = r.choices[0].message.content
        results.append(res)
        print(res)
        
    except Exception as e:
        Errors += 1
        # Error_L.append(i)
        print(e)
    


```json
[19, 20, 43, 44, 47, 48, 49, 67, 68, 98]
```
```json
[20, 19, 43, 44, 47, 61, 66, 100, 101, 283]
```
```json
[21, 104, 105, 106, 108, 110, 113, 115, 116, 129]
```
```json
[21, 22, 23, 112, 197, 238, 239, 425, 501, 506]
```
```json
[24, 25, 26, 23, 37, 301, 302, 400, 301, 302]
```
```json
[24, 302, 403, 404, 301, 37, 309, 400, 302, 37]
```
```json
[25, 24, 37, 301, 302, 309, 403]
```
```json
[46, 45, 27, 26, 28, 258, 260, 261, 310, 310]
```
```json
[27, 26, 260, 258, 215, 282, 283, 287, 296, 282]
```
```json
[30, 31, 22, 25, 21, 34, 173, 399, 284, 434]
```
```json
[31, 30, 32, 25, 116, 278, 315, 352, 357, 361]
```
```json
[32, 64, 73, 198, 199, 275, 403, 417, 418, 31]
```
```json
[34, 35, 32, 36, 31, 39, 37, 32, 33, 31]
```
```json
[37, 35, 36, 39, 34, 32, 60, 57, 58, 55]
```
```json
[36, 35, 37, 39, 34, 55, 57, 58, 60, 201]
```
```json
[37, 39, 32, 31, 60, 64, 73, 198, 199, 275]
```
```json
[39, 37, 36, 32, 55, 58, 60, 73, 198, 199]
```
```json
[43, 44, 47, 51, 52, 53, 54, 63, 

In [14]:
import json
true_result = []
for res in results:
    temp = res.replace("`", "").replace("json", "").replace("\"", "")
    true_result.append(json.loads(temp))

top1 = 0
top3 = 0
top5 = 0
top10 = 0
# top20 = 0
# top50 = 0
# top100 = 0

page_distances = [] # the distances of the correct page and retrieved page
scores = [] # the score of the corresponding page and question
correct_rank = [] # the index of the correct page in the retrieved results.

for i in range(len(data)):
    page_number = data[i]['page']
    reranked = true_result[i]
    # retrived_page_rank = [list(data[i]['question_retrived_page_rank'][d].keys())[0] for d in range(len(data[i]['question_retrived_page_rank']))]
    # correct_page_rank = retrived_page_rank.index(page_number)
    # correct_rank.append(correct_page_rank)
    # page_distances.append(abs(page_number - retrived_page_rank[0]))
    # scores.append(cos_sim(data[i]['question_embedding'], data[i]['page_embedding']))
    
    if page_number in reranked[:1]: top1 += 1
    if page_number in reranked[:3]: top3 += 1
    if page_number in reranked[:5]: top5 += 1
    if page_number in reranked[:10]: top10 += 1
    # if page_number in retrived_page_rank[:20]: top20 += 1
    # if page_number in retrived_page_rank[:50]: top50 += 1
    # if page_number in retrived_page_rank[:100]: top100 += 1

In [15]:
print(top1/477)
print(top3/477)
print(top5/477)
print(top10/477)

0.5157232704402516
0.8176100628930818
0.870020964360587
0.949685534591195
